In [28]:
#imports
import numpy as np
import pandas as pd
from scipy import sparse as sparse
from scipy.sparse import linalg as splinalg
from sys import argv
import matplotlib.pyplot as plt
import pickle as pl
import warnings
from scipy.integrate import odeint, solve_ivp
warnings.filterwarnings("ignore")
import collections
import os
from tqdm import tqdm
import random


In [29]:
#imports
import torch
import torch.nn as nn
from torch.nn  import functional as F
#use gpu if available
device = torch.device('mps' if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [30]:
# read it in to inspect it
with open('../words_250000_train.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

c = collections.Counter(text)
sorted_letter_count = c.most_common() 
train_word_list = np.array(text.split('\n'), dtype = str)[:227200]
train_len = np.array([len(itr) for itr in train_word_list])

print(len(train_word_list), train_word_list)

length of dataset in characters:  2352046
227200 ['aaa' 'aaaaaa' 'aaas' ... 'zwart' 'zweig' 'zwickau']


In [31]:
letter_notpresent = []
for itr in tqdm(train_word_list):

    temp = np.array([list(np.arange(26))]*len(itr)).T
    to_sub = np.array([ord(itr1)-97 for itr1 in itr])
    mask = temp - to_sub[None, :]
    mask = np.prod(mask, axis = 1)
    mask = mask != 0
    
    letter_notpresent.append(np.arange(26)[mask])
    

train_word_list = np.array([ list(w.ljust(40)) for w in train_word_list])

 
    


len_mask = torch.tensor([np.concatenate((np.zeros(train_len[itr]), np.ones(30-train_len[itr]))) for itr in range(len(train_len))])
print(len_mask.shape)

print(train_word_list[:10])
print(train_len[:10])
print(min(train_len), max(train_len))
print(train_word_list[np.argmin(train_len)])
print(train_word_list[np.argmax(train_len)])
# get the vocabulary
chars = sorted(list(set(text))) # vocab
chars[0]=' '
chars.insert(1, '_')
chars.insert(len(chars), ',')
vocab_size = len(chars)
print(f'Vocabulary size: {vocab_size}')

# print vocab
print(chars)
print(''.join(chars))

100%|███████████████████████████████| 227200/227200 [00:09<00:00, 22922.94it/s]


torch.Size([227200, 30])
[['a' 'a' 'a' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ']
 ['a' 'a' 'a' 'a' 'a' 'a' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ']
 ['a' 'a' 'a' 's' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ']
 ['a' 'a' 'c' 'h' 'e' 'n' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ']
 ['a' 'a' 'e' 'e' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ']
 ['a' 'a' 'g' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 

In [32]:
print(len_mask[:5])
print(train_word_list[:5])

tensor([[0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=torch.float64)
[['a' 'a' 'a' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ']
 ['a' 'a' 'a' 'a' 'a' 'a' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 

Vocabulary is the set of unique characters / words in the dataset

In [33]:
# create a mapping from characters to integers and vice versa

stoi = {ch:i for i, ch in enumerate(chars)} # abbrev for string to integer
itos = {i:ch for ch, i in stoi.items()}

# encoder: takes a string and returns a list of integers
encode = lambda s: [stoi[c] for c in s]

# decoder: takes a list of integers and returns a string
decode = lambda l: ''.join([itos[i] for i in l])

#unit test
print(encode('hi_there,'.ljust(40)))
print(decode(encode('hi_there')))
print(stoi)

[9, 10, 1, 21, 9, 6, 19, 6, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
hi_there
{' ': 0, '_': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27, ',': 28}


### Data loader

In [34]:
def get_batch_new(train_word_list, ix, train_len, avg_num_letters = 7, max_wrong_guesses = 6):
    # create a batch using the list of indices, ix
    batch_size = len(ix)
    letter_array = np.arange(26)
    p = avg_num_letters/26
    temp_select = np.random.random((26, batch_size))

    y = torch.zeros((batch_size, 40), dtype = torch.int)
    x = torch.zeros((batch_size, 40), dtype = torch.int)
    # x, y = x.to(device), y.to(device)
    
    for itr in range(batch_size):
        y[itr] = torch.tensor(encode(train_word_list[ix[itr]]))
        x[itr] = y[itr]
        letter_select = torch.tensor(letter_array[temp_select[:, itr] < p] + 2)
        letter_unselect = torch.tensor(np.random.choice(letter_notpresent[ix[itr]] + 2,
                                                        size =  min(26 - len(letter_select), np.random.randint(0, max_wrong_guesses)), replace = False))
        mask = torch.isin(x[itr], letter_select)
        x[itr] = x[itr].masked_fill(mask, 1)
        len_curr = train_len[ix[itr]]
        x[itr][len_curr] = 28
        for itr1, letter in enumerate(letter_unselect):
            if itr1+len_curr + 1 >= 40: break
            x[itr][itr1 + len_curr + 1] = letter
            

    
    return x.to(device), y.to(device)


def batch_iterator(train_word_list, train_len, batch_size, avg_num_letters, max_wrong_guesses):
    n = len(train_word_list)
    indices = np.random.permutation(n)  # shuffle once per epoch
    flag = False
    if avg_num_letters==-1:
        flag = True
    for start in range(0, n, batch_size):
        end = start + batch_size
        batch_idx = indices[start:end]
        if flag:
            avg_num_letters = random.randint(7, 15)
        X, y = get_batch_new(train_word_list, batch_idx, train_len, avg_num_letters, max_wrong_guesses)
        yield X, y



#This piece of code checks if the batch iterator is working 

for epoch in range(2):
    print('Epoch Number: ', epoch)
    for i in range(2):
        for X, y in  batch_iterator(train_word_list[:10], train_len[:10], 5, 3, 10):
            for j in range(5):
                print(decode(list(y[j].cpu().detach().numpy())))
        

Epoch Number:  0
aag                                     
aaas                                    
aahs                                    
aaaaaa                                  
aaee                                    
aahed                                   
aaa                                     
aachen                                  
aal                                     
aalesund                                
aag                                     
aal                                     
aaa                                     
aahed                                   
aaee                                    
aachen                                  
aaas                                    
aahs                                    
aaaaaa                                  
aalesund                                
Epoch Number:  1
aachen                                  
aal                                     
aaa                                     
aaas                   

### Part 1: Define a decoder only head to perform scaled dot product attention.

Consider an input $x \in \mathbb{R}^{B \times T \times C}$ to the attention head, where $B$ is the batch size, $T$ is the context length and $C$ is the embedding dimension (input dimension from the last layer).

Then, the attention head of size $d$ outputs $z \in \mathbb{R}^{B \times T \times d}$

$$ z = softmax \left( \frac{Q K^T}{\sqrt{d}} \right) V $$

where $K, Q, V$ are linear (and not affine) transformations of the input

$$ K = k x, \text{ with } k \in \mathbb{R}^{d \times C} $$
$$ Q = q x, \text{ with } q \in \mathbb{R}^{d \times C} $$
$$ V = v x, \text{ with } v \in \mathbb{R}^{d \times C} $$

Define a nn.Module class to implement a decoder only head.


In [35]:
class Head(nn.Module):
    """Description: decoder only scaled dot product attention"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        """
        Description:
            Initializes various layers used in the forward function
            Note: runs when an object of Head is created
        Input:
            n_embd (C): The embedding dimension
            n_head (d): Hidden dimension of the single head, i.e., key.shape = query.shape = value.shape = (n_embd, head_size)
        """
        # use linear layers to define key, query and value matrices
        # bias is usually not used in nn.Linear in the attention blocks; use bias = False
        self.key = nn.Linear(n_embd, n_head, bias = False, device = device)
        self.query = nn.Linear(n_embd, n_head, bias = False, device = device)
        self.value = nn.Linear(n_embd, n_head, bias = False, device = device)


    def forward(self, x):
        """
        Description:
            Forward pass of the attention head
        Input:
            x of shape (B, T, C)
        Output:
            z of shape (B, T, d)
        """
        x=x.to(device)
        B, T, C = x.shape # B: batch size; T: block_size; C: embedding dim
        K = self.key(x)    # (B, T, d)
        Q = self.query(x)  # (B, T, d)
        # compute scaled dot product attention scores w = q @ k / sqrt(d)
        W = torch.einsum('btd,bud->btu', Q, K) # (B, T, d) @ (B, T, d) -> (B, T, T)
        # Mask out the attention scores such that the padded values are ignored
        mask = torch.ones((B, T), device = device).masked_fill(torch.sum(x, axis = -1)==0, float('-inf')) # (B, T, T)
        
        # apply a softmax along the last dim;
        # use nn.functional.softmax;
        # Note: nn.functional is imported as F
        W = F.softmax(W + mask[:, None, :], dim=2) # (B, T, T)
#         print(W[:, 0,:])
        # perform the weighted aggregation of the values: V = v @ x
        V = self.value(x) # (B, T, d)
        # output w @ v
        out = W @ V # (B, T, T) @ (B, T, d) -> (B, T, d)
        return out

In [36]:
x, y = get_batch_new(train_word_list, [10, 20], train_len, 2)
print(x)
print(y)
print(y.masked_fill(x>1, 0))
print(x)
test_embd = nn.Embedding(28, 28, padding_idx=0)
test_embd = test_embd.to(device)
x = test_embd(x).to(device)
head = Head(28, 56)
out = head(x)
print(out.shape)
print(train_word_list[10], train_word_list[20])

tensor([[ 2,  2, 13, 10, 10, 20, 28,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0],
        [ 2,  2,  1,  6, 15, 28, 18, 24, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0]], device='mps:0', dtype=torch.int32)
tensor([[ 2,  2, 13, 10, 10, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0],
        [ 2,  2, 19,  6, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0]], device='mps:0', dtype=torch.int32)
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  

### Part 2: Define multi-head attention module

Given input $x \in \mathbb{R}^{B \times T \times C}$ as before, define multiple heads of attention that work in parallel. Use class Head() defined above to create a list of attention heads using nn.ModuleList()

Each attention head returns $z \in \mathbb{R}^{B \times T \times d}$

Using a linear layer, project the concatentated output of attention heads back to the embedding dimension $C$.

In [37]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size, embd_size):
        super().__init__()
        """
        Description: Performs multi-head attention followed by a projection
        Input:
            num_heads: number of attention heads
            head_size (d): size of each attention head
        """
        self.heads = nn.ModuleList([Head(embd_size, head_size) for i in range(num_heads)]) # Define n_heads copies of Head() of size head_size as a list
        self.proj = nn.Linear(num_heads*head_size, embd_size, bias = False) # projection layer using nn.Linear

    def forward(self, x):
        """"
        Description: Forward pass of multi head attention
        Input: x of shape (B, T, C)
        """
        multihead_out = []
        for i, l in enumerate(self.heads):
            multihead_out.append(l(x))
        out = torch.cat(multihead_out, dim = 2)

        #project the output using a linear
        out = self.proj(out)
        return out

### Part 3: Implement a simple two layer ReLU FCN $f: \mathbb{R}^n \to \mathbb{R}^n$ with hidden layer dimension $h = 4 \times n$ .

The network should perform the following operations:

1. Linear: $\mathbb{R}^{n \to 4n}$
2. ReLU: $\mathbb{R}^{4n \to 4n}$
3. Linear: $\mathbb{R}^{n \to n}$

In [38]:
class FeedFoward(nn.Module):
    """ A simple one hidden layer ReLU block followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.layer1 = nn.Linear(n_embd, 4*n_embd)
        self.layer2 = nn.Linear(4*n_embd, n_embd)
        return
        """
        Description: Linear -> ReLU -> Linear
        """

    def forward(self, x):
        "Forward pass of the network"
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x

### Part 4: Implement a Transformer block consisting of multi-head attention followed by feedforward computation

Using the MultiHeadAttention() and Feedforward() classes, define a Transformer block to perform:

1. LayerNorm in the embedding dimension using nn.LayerNorm(n_embd)
2. Multi-head attention with n_head heads of size head_size
3. Add Residual from input
4. LayerNorm in the embedding dimension using nn.LayerNorm(n_embd)
5. Feedforward layer: Linear -> ReLU -> Linear
6. Add Residual from Step 3

In [39]:
class Block(nn.Module):
    """ Transformer block: Multi head attention followed by feed forward followed by LayerNorm """

    def __init__(self, n_embd, n_head, head_size = 100):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        """
        Description: Transformer block
        Input:
            n_embd: embedding dimension of the input
            n_head: number of attention heads
        """
        head_size = n_embd // n_head
        self.L1 = nn.LayerNorm(n_embd)
        self.L2 = MultiHeadAttention(n_head, head_size, n_embd)
        self.L3 = FeedFoward(n_embd)
        return

    def forward(self, x):
        "Forward pass of the attention block"
        residual = x
        x = self.L2(x)
        x = x + residual
        residual = x
        x = self.L3(x)
        x = x + residual
        return x


### Part 5: The full (decoder only) Transformer

Finally, lets put all of it together to construct the full decoder only transformer. Consider the tokenized input $x \in \mathbb{R}^{B \times T \times V}$, where $B$ is the batch size, $T$ is the context length and $V$ is the vocabulary size.

The model should perform the following computations:

1. T = Token embedding(x) $ \in \mathbb{R}^{B \times T \times C}$, where $C$ is the embedding dimension
2. P = Positional embedding(x) $\in \mathbb{R}^{T \times C}$ .For simplicity, we will use the index of the sequence as the positional embedding.
3. X = T + P  $ \in \mathbb{R}^{B \times T \times C}$
4. n_layers of Transformer blocks using the Block() module.
Use nn.Sequential() for stacking mulitple layers
5. LayerNorm in the embeding dimension
6. Linear transformation: $\mathbb{R}^{C} \to \mathbb{R}^{V}$






In [40]:
def get_rotary_embedding(dim, seq_len, device):
    """Computes rotary position embeddings using torch built-in functions."""
    theta = 10000.0 ** (-torch.arange(0, dim, 2, dtype=torch.float32, device=device) / dim)
    seq_idx = torch.arange(seq_len, device=device, dtype=torch.float32).unsqueeze(1)
    emb = torch.matmul(seq_idx, theta.unsqueeze(0))
    return torch.cat([emb.sin(), emb.cos()], dim=-1)

def apply_rotary_embedding(x, rotary_emb):
    """Applies rotary embedding using PyTorch operations."""
    x_reshaped = x.view(*x.shape[:-1], x.shape[-1] // 2, 2)
    cos_emb, sin_emb = rotary_emb.chunk(2, dim=-1)
    x1, x2 = x_reshaped.unbind(-1)
    x_rot = torch.stack((x1 * cos_emb - x2 * sin_emb, x1 * sin_emb + x2 * cos_emb), dim=-1)
    return x_rot.flatten(-2)

# Example usage

dim = 64  # Embedding dimension
seq_len = 128  # Sequence length
batch_size = 2  # Example batch size

# Generate random input tensor
x = torch.randn(batch_size, seq_len, dim, device=device)

# Compute rotary embeddings
rotary_emb = get_rotary_embedding(dim, seq_len, device)

# Apply rotary embeddings
x_rotated = apply_rotary_embedding(x, rotary_emb)

print(x_rotated.shape)  # Should be (batch_size, seq_len, dim)

torch.Size([2, 128, 64])


In [41]:
class Transformer(nn.Module):
    """
    Description: Decoder only transformer model"""
    def __init__(self, vocab_size, block_size, n_embd, n_layers, n_head):
        super().__init__()
        """
        Description: Decoder only transformer model
        Input:
            vocab_size (V) : Vocabulary dimension
            block_size (T): Context length
            n_embd (C): Embedding dimension
            n_layers: number of layers of Transformer blocks
            n_head : number of heads in multi-head attention
        """
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd, padding_idx = 0) 
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for itre in range(n_layers)])
        self.layer_norm = nn.LayerNorm(n_embd) # Layer norm in the embedding dimension
        self.linear_head = nn.Linear(n_embd, vocab_size) # (C -> V)

    ### DO NOT MODIFY BEYOND THIS
    def forward(self, idx, targets=None):
        """
        Description: Forward pass of transformer
        Inputs:
            idx: The tokenized input sequence
            targets (optional): the tokenized target sequence
        """
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        _, _, C = tok_emb.shape
        rotary_emb = get_rotary_embedding(C, block_size, device)
        x = apply_rotary_embedding(tok_emb, rotary_emb).masked_fill(tok_emb==0, 0)
        x = self.blocks(x) # (B,T,C)
        x = self.layer_norm(x) # (B,T,C)
        logits = self.linear_head(x) # (B,T, V)

        # computes loss if targets are provided
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.masked_fill(idx > 1 , 0)
            targets = targets.view(B*T).type(torch.LongTensor).to(device)
            
            loss = F.cross_entropy(logits, targets, ignore_index = 0)

        return logits, loss





## Loss Function

Estimates training and test loss

In [42]:
### DO NOT MODIFY THIS
# Estimates loss
def estimate_loss(frac = 0.27):
    model.eval()
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        X, Y = get_batch(train_word_list, train_len, batch_size, frac)
        logits, loss =  model(X, Y)
        losses[k] = loss.item()
    mean_loss = losses.mean()
    model.train()
    return mean_loss

## Training and Hyperparameters

In [55]:
block_size = 40 # maximum context length
n_embd = 127
n_head = 16
n_layers = 5
dropout = 0.0
print(vocab_size)

29


In [56]:
def printModel(model):
    print("Model's state_dict:")
    for param_tensor in model.state_dict():
        print(param_tensor, "\t", model.state_dict()[param_tensor].size())

torch.manual_seed(1090)
model = Transformer(vocab_size, block_size, n_embd+1, n_layers, n_head)
model = model.to(device)
# printModel(model)

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

0.995229 M parameters


In [48]:
learning_rate = 1e-4
batch_size = 800
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=0.01)
num_epochs = 30
for epoch in range(num_epochs):
    epoch_losses = []
    model.train()
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 15, 6):
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    # After every epoch see the loss on training set
    model.eval()
    train_loss = 0
    niter = 0
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 15, 6):
        logits, loss = model(xb, yb)
        train_loss += loss.item()
        niter += 1
    train_loss = train_loss/niter
        
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Avg batch loss: {np.mean(epoch_losses):.4f} - "
          f"Full train loss: {train_loss:.4f}")

    if (epoch+1)%5==0:
        torch.save(model.state_dict(), f'model_{n_embd}_{n_head}_{n_layers}_epoch_{epoch+1}')

RuntimeError: MPS backend out of memory (MPS allocated: 5.97 GB, other allocations: 825.25 MB, max allowed: 6.77 GB). Tried to allocate 31.25 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [299]:
learning_rate = 1e-4
batch_size = 400
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=0.01)
num_epochs = 20
for epoch in range(30, 30 + num_epochs):
    epoch_losses = []
    model.train()
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 15, 6):
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    # After every epoch see the loss on training set
    model.eval()
    train_loss = 0
    niter = 0
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 15, 6):
        logits, loss = model(xb, yb)
        train_loss += loss.item()
        niter += 1
    train_loss = train_loss/niter
        
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Avg batch loss: {np.mean(epoch_losses):.4f} - "
          f"Full train loss: {train_loss:.4f}")

    if (epoch+1)%5==0:
        torch.save(model.state_dict(), f'model_{n_embd}_{n_head}_{n_layers}_epoch_{epoch+1}')

Epoch 31/20 - Avg batch loss: 2.0212 - Full train loss: 2.0169
Epoch 32/20 - Avg batch loss: 2.0185 - Full train loss: 2.0152
Epoch 33/20 - Avg batch loss: 2.0166 - Full train loss: 2.0115
Epoch 34/20 - Avg batch loss: 2.0116 - Full train loss: 2.0114
Epoch 35/20 - Avg batch loss: 2.0097 - Full train loss: 2.0075
Epoch 36/20 - Avg batch loss: 2.0057 - Full train loss: 2.0029
Epoch 37/20 - Avg batch loss: 2.0040 - Full train loss: 2.0038
Epoch 38/20 - Avg batch loss: 2.0010 - Full train loss: 1.9992
Epoch 39/20 - Avg batch loss: 1.9988 - Full train loss: 1.9973
Epoch 40/20 - Avg batch loss: 1.9955 - Full train loss: 1.9984
Epoch 41/20 - Avg batch loss: 1.9937 - Full train loss: 1.9912
Epoch 42/20 - Avg batch loss: 1.9927 - Full train loss: 1.9883
Epoch 43/20 - Avg batch loss: 1.9895 - Full train loss: 1.9863
Epoch 44/20 - Avg batch loss: 1.9878 - Full train loss: 1.9849
Epoch 45/20 - Avg batch loss: 1.9874 - Full train loss: 1.9840
Epoch 46/20 - Avg batch loss: 1.9838 - Full train loss:

In [300]:
learning_rate = 3e-5
batch_size = 400
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=0.01)
num_epochs = 20
for epoch in range(50, 50 + num_epochs):
    epoch_losses = []
    model.train()
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 10, 6):
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    # After every epoch see the loss on training set
    model.eval()
    train_loss = 0
    niter = 0
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 15, 6):
        logits, loss = model(xb, yb)
        train_loss += loss.item()
        niter += 1
    train_loss = train_loss/niter
        
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Avg batch loss: {np.mean(epoch_losses):.4f} - "
          f"Full train loss: {train_loss:.4f}")

    if (epoch+1)%5==0:
        torch.save(model.state_dict(), f'model_{n_embd}_{n_head}_{n_layers}_epoch_{epoch+1}')

Epoch 51/20 - Avg batch loss: 1.6564 - Full train loss: 1.9644
Epoch 52/20 - Avg batch loss: 1.6526 - Full train loss: 1.9631
Epoch 53/20 - Avg batch loss: 1.6487 - Full train loss: 1.9603
Epoch 54/20 - Avg batch loss: 1.6471 - Full train loss: 1.9626
Epoch 55/20 - Avg batch loss: 1.6450 - Full train loss: 1.9604
Epoch 56/20 - Avg batch loss: 1.6444 - Full train loss: 1.9602
Epoch 57/20 - Avg batch loss: 1.6427 - Full train loss: 1.9583
Epoch 58/20 - Avg batch loss: 1.6411 - Full train loss: 1.9576
Epoch 59/20 - Avg batch loss: 1.6381 - Full train loss: 1.9578
Epoch 60/20 - Avg batch loss: 1.6405 - Full train loss: 1.9576
Epoch 61/20 - Avg batch loss: 1.6392 - Full train loss: 1.9566
Epoch 62/20 - Avg batch loss: 1.6360 - Full train loss: 1.9542
Epoch 63/20 - Avg batch loss: 1.6339 - Full train loss: 1.9566
Epoch 64/20 - Avg batch loss: 1.6358 - Full train loss: 1.9531
Epoch 65/20 - Avg batch loss: 1.6340 - Full train loss: 1.9538
Epoch 66/20 - Avg batch loss: 1.6297 - Full train loss:

In [17]:
state = torch.load('model_127_16_5_epoch_70')
model = Transformer(vocab_size, block_size, n_embd+1, n_layers, n_head)
model.load_state_dict(state)
model = model.to(device)

In [18]:
learning_rate = 3e-5
batch_size = 400
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=0.01)
num_epochs = 20
for epoch in range(70, 70 + num_epochs):
    epoch_losses = []
    model.train()
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 8, 6):
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    # After every epoch see the loss on training set
    model.eval()
    train_loss = 0
    niter = 0
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 15, 6):
        logits, loss = model(xb, yb)
        train_loss += loss.item()
        niter += 1
    train_loss = train_loss/niter
        
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Avg batch loss: {np.mean(epoch_losses):.4f} - "
          f"Full train loss: {train_loss:.4f}")

    if (epoch+1)%5==0:
        torch.save(model.state_dict(), f'model_{n_embd}_{n_head}_{n_layers}_epoch_{epoch+1}')

Epoch 71/20 - Avg batch loss: 1.5024 - Full train loss: 1.9533
Epoch 72/20 - Avg batch loss: 1.5041 - Full train loss: 1.9565
Epoch 73/20 - Avg batch loss: 1.5041 - Full train loss: 1.9550
Epoch 74/20 - Avg batch loss: 1.5031 - Full train loss: 1.9525
Epoch 75/20 - Avg batch loss: 1.4986 - Full train loss: 1.9544
Epoch 76/20 - Avg batch loss: 1.4978 - Full train loss: 1.9515
Epoch 77/20 - Avg batch loss: 1.4978 - Full train loss: 1.9521
Epoch 78/20 - Avg batch loss: 1.4986 - Full train loss: 1.9495
Epoch 79/20 - Avg batch loss: 1.4950 - Full train loss: 1.9511
Epoch 80/20 - Avg batch loss: 1.4960 - Full train loss: 1.9518
Epoch 81/20 - Avg batch loss: 1.4942 - Full train loss: 1.9507
Epoch 82/20 - Avg batch loss: 1.4880 - Full train loss: 1.9514
Epoch 83/20 - Avg batch loss: 1.4918 - Full train loss: 1.9502
Epoch 84/20 - Avg batch loss: 1.4902 - Full train loss: 1.9485
Epoch 85/20 - Avg batch loss: 1.4895 - Full train loss: 1.9503
Epoch 86/20 - Avg batch loss: 1.4885 - Full train loss:

In [20]:
learning_rate = 1e-5
batch_size = 400
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=0.01)
num_epochs = 20
for epoch in range(90, 90 + num_epochs):
    epoch_losses = []
    model.train()
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 8, 6):
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    # After every epoch see the loss on training set
    model.eval()
    train_loss = 0
    niter = 0
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 15, 6):
        logits, loss = model(xb, yb)
        train_loss += loss.item() 
        niter += 1
    train_loss = train_loss/niter
        
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Avg batch loss: {np.mean(epoch_losses):.4f} - "
          f"Full train loss: {train_loss:.4f}")

    if (epoch+1)%5==0:
        torch.save(model.state_dict(), f'model_{n_embd}_{n_head}_{n_layers}_epoch_{epoch+1}')

Epoch 91/20 - Avg batch loss: 1.4779 - Full train loss: 1.9424
Epoch 92/20 - Avg batch loss: 1.4769 - Full train loss: 1.9454
Epoch 93/20 - Avg batch loss: 1.4791 - Full train loss: 1.9431
Epoch 94/20 - Avg batch loss: 1.4753 - Full train loss: 1.9431
Epoch 95/20 - Avg batch loss: 1.4770 - Full train loss: 1.9417
Epoch 96/20 - Avg batch loss: 1.4758 - Full train loss: 1.9425
Epoch 97/20 - Avg batch loss: 1.4734 - Full train loss: 1.9409
Epoch 98/20 - Avg batch loss: 1.4728 - Full train loss: 1.9414
Epoch 99/20 - Avg batch loss: 1.4752 - Full train loss: 1.9390
Epoch 100/20 - Avg batch loss: 1.4761 - Full train loss: 1.9407
Epoch 101/20 - Avg batch loss: 1.4750 - Full train loss: 1.9399
Epoch 102/20 - Avg batch loss: 1.4734 - Full train loss: 1.9415
Epoch 103/20 - Avg batch loss: 1.4716 - Full train loss: 1.9411
Epoch 104/20 - Avg batch loss: 1.4729 - Full train loss: 1.9426
Epoch 105/20 - Avg batch loss: 1.4726 - Full train loss: 1.9421
Epoch 106/20 - Avg batch loss: 1.4710 - Full trai

In [21]:
learning_rate = 1e-5
batch_size = 400
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=0.01)
num_epochs = 20
for epoch in range(110, 110 + num_epochs):
    epoch_losses = []
    model.train()
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 10, 6):
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    # After every epoch see the loss on training set
    model.eval()
    train_loss = 0
    niter = 0
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 15, 6):
        logits, loss = model(xb, yb)
        train_loss += loss.item() 
        niter += 1
    train_loss = train_loss/niter
        
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Avg batch loss: {np.mean(epoch_losses):.4f} - "
          f"Full train loss: {train_loss:.4f}")

    if (epoch+1)%5==0:
        torch.save(model.state_dict(), f'model_{n_embd}_{n_head}_{n_layers}_epoch_{epoch+1}')

Epoch 111/20 - Avg batch loss: 1.5991 - Full train loss: 1.9378
Epoch 112/20 - Avg batch loss: 1.5993 - Full train loss: 1.9358
Epoch 113/20 - Avg batch loss: 1.6001 - Full train loss: 1.9364
Epoch 114/20 - Avg batch loss: 1.5974 - Full train loss: 1.9343
Epoch 115/20 - Avg batch loss: 1.5988 - Full train loss: 1.9334
Epoch 116/20 - Avg batch loss: 1.5981 - Full train loss: 1.9358
Epoch 117/20 - Avg batch loss: 1.5985 - Full train loss: 1.9346
Epoch 118/20 - Avg batch loss: 1.5962 - Full train loss: 1.9339
Epoch 119/20 - Avg batch loss: 1.5964 - Full train loss: 1.9346
Epoch 120/20 - Avg batch loss: 1.5952 - Full train loss: 1.9327
Epoch 121/20 - Avg batch loss: 1.5965 - Full train loss: 1.9349
Epoch 122/20 - Avg batch loss: 1.5955 - Full train loss: 1.9309
Epoch 123/20 - Avg batch loss: 1.5965 - Full train loss: 1.9320
Epoch 124/20 - Avg batch loss: 1.5942 - Full train loss: 1.9328
Epoch 125/20 - Avg batch loss: 1.5950 - Full train loss: 1.9317
Epoch 126/20 - Avg batch loss: 1.5949 - 

In [26]:
state = torch.load('model_127_16_5_epoch_130')
model = Transformer(vocab_size, block_size, n_embd+1, n_layers, n_head)
model.load_state_dict(state)
model = model.to(device)

In [27]:
learning_rate = 1e-5
batch_size = 400
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=0.01)
num_epochs = 20
for epoch in range(130, 130 + num_epochs):
    epoch_losses = []
    model.train()
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 8, 6):
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    # After every epoch see the loss on training set
    model.eval()
    train_loss = 0
    niter = 0
    for xb, yb in  batch_iterator(train_word_list, train_len, batch_size, 15, 6):
        logits, loss = model(xb, yb)
        train_loss += loss.item() 
        niter += 1
    train_loss = train_loss/niter
        
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Avg batch loss: {np.mean(epoch_losses):.4f} - "
          f"Full train loss: {train_loss:.4f}")

    if (epoch+1)%5==0:
        torch.save(model.state_dict(), f'model_{n_embd}_{n_head}_{n_layers}_epoch_{epoch+1}')

Epoch 131/20 - Avg batch loss: 1.4653 - Full train loss: 1.9348
Epoch 132/20 - Avg batch loss: 1.4610 - Full train loss: 1.9350
Epoch 133/20 - Avg batch loss: 1.4641 - Full train loss: 1.9380
Epoch 134/20 - Avg batch loss: 1.4625 - Full train loss: 1.9334
Epoch 135/20 - Avg batch loss: 1.4631 - Full train loss: 1.9357
Epoch 136/20 - Avg batch loss: 1.4649 - Full train loss: 1.9319
Epoch 137/20 - Avg batch loss: 1.4608 - Full train loss: 1.9327
Epoch 138/20 - Avg batch loss: 1.4609 - Full train loss: 1.9338
Epoch 139/20 - Avg batch loss: 1.4639 - Full train loss: 1.9336
Epoch 140/20 - Avg batch loss: 1.4632 - Full train loss: 1.9336
Epoch 141/20 - Avg batch loss: 1.4609 - Full train loss: 1.9356
Epoch 142/20 - Avg batch loss: 1.4609 - Full train loss: 1.9358
Epoch 143/20 - Avg batch loss: 1.4637 - Full train loss: 1.9325
Epoch 144/20 - Avg batch loss: 1.4610 - Full train loss: 1.9348
Epoch 145/20 - Avg batch loss: 1.4593 - Full train loss: 1.9325
Epoch 146/20 - Avg batch loss: 1.4613 - 